In [12]:
import pandas as pd
import numpy as np
import joblib
import json
import os
import warnings
import plotly.express as px
import plotly.graph_objects as go
import onnxruntime as ort
from sklearn.metrics import (
    accuracy_score, f1_score, recall_score, 
    confusion_matrix, roc_curve, auc, 
    precision_recall_curve, average_precision_score
)
from sklearn.model_selection import train_test_split
from sklearn.inspection import permutation_importance

warnings.filterwarnings('ignore')

# 분석에 필요한 파일들과 이미지를 저장할 경로를 지정합니다
metrics_path = '../data/model_metrics.json'
ml_model_path = '../models/spotify_churn_model.pkl'
dl_model_path = '../models/spotify_dl_model.onnx'
dl_preprocessor_path = '../models/dl_preprocessor.pkl'
data_path = '../data/spotify_churn_dataset.csv'
image_dir = '../02_training_report/images/'

if not os.path.exists(image_dir):
    os.makedirs(image_dir)

# 모델별 성능 지표를 불러와 데이터프레임으로 정리합니다
with open(metrics_path, 'r', encoding='utf-8') as f:
    metrics = json.load(f)

df_metrics = pd.DataFrame(metrics).T.reset_index().rename(columns={
    'index': '모델',
    'Accuracy': '정확도',
    'Recall': '재현율',
    'F1-Score': 'F1 스코어'
})

# 모델 이름을 리포트용 한글 이름으로 변경합니다
df_metrics['모델'] = df_metrics['모델'].replace({
    'Deep Learning (DNN)': '딥러닝(DNN)', 
    'RandomForest': '머신러닝(RF)',
    'Deep Learning (PyTorch_ONNX)': '딥러닝(DNN)'
})

# 선명한 색상을 사용하고 막대 위에 소수점 4자리까지 수치를 표시합니다
fig_performance = px.bar(
    df_metrics, x='모델', y=['정확도', '재현율', 'F1 스코어'],
    barmode='group', text_auto='.4f', 
    title='모델별 주요 성능 지표 비교 (정확도, 재현율, F1)',
    labels={'value': '점수', 'variable': '지표'},
    color_discrete_sequence=px.colors.qualitative.Vivid
)
fig_performance.update_traces(textposition='outside')
fig_performance.update_layout(yaxis_range=[0, 1.2])
fig_performance.write_image(f'{image_dir}model_performance_bar.png')
fig_performance.show()

# 테스트 데이터를 로드하고 학습 시와 동일한 파생 변수를 생성합니다
df = pd.read_csv(data_path)
if 'user_id' in df.columns: df = df.drop(columns=['user_id'])
df['ad_burden'] = df['ads_listened_per_week'] / (df['listening_time'] + 1)
df['satisfaction_score'] = df['songs_played_per_day'] * (1 - df['skip_rate'])
df['time_per_song'] = df['listening_time'] / (df['songs_played_per_day'] + 1)

X = df.drop(columns=['is_churned'])
y = df['is_churned']
_, X_test, _, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# 학습된 모델과 전처리기를 불러옵니다
ml_model = joblib.load(ml_model_path)
dl_prep = joblib.load(dl_preprocessor_path)
ort_session = ort.InferenceSession(dl_model_path)

# 사이킷런의 순열 중요도 함수와 호환되도록 딥러닝 모델을 감싸는 클래스를 만듭니다
class DLWrapper:
    def __init__(self, session, prep, threshold):
        self.session = session
        self.prep = prep
        self.threshold = threshold
        self._estimator_type = "classifier"
        self.classes_ = np.array([0, 1])
    
    def fit(self, X, y): return self
    
    def predict(self, X):
        # 입력 데이터를 전처리하고 ONNX 모델로 추론을 수행합니다
        X_proc = self.prep.transform(X).astype(np.float32)
        input_name = self.session.get_inputs()[0].name
        probs = self.session.run(None, {input_name: X_proc})[0].flatten()
        return (probs >= self.threshold).astype(int)

# 딥러닝 모델에 내장된 중요도 속성이 없으므로 순열 중요도 기법으로 각 특성의 기여도를 산출합니다
print("두 모델의 특성 중요도를 계산하고 있습니다. 데이터 양에 따라 시간이 조금 소요됩니다...")
dl_threshold = metrics.get('Deep Learning (DNN)', {}).get('Best Threshold', 0.5)
dl_model_wrapped = DLWrapper(ort_session, dl_prep, dl_threshold)

r_ml = permutation_importance(ml_model, X_test, y_test, n_repeats=5, random_state=42, scoring='f1')
r_dl = permutation_importance(dl_model_wrapped, X_test, y_test, n_repeats=5, random_state=42, scoring='f1')

# 머신러닝(RF) 중요도 시각화 - Spotify 초록색 테마
df_ml_imp = pd.DataFrame({'특성': X_test.columns, '중요도': r_ml.importances_mean}).sort_values(by='중요도', ascending=True)
fig_ml_imp = px.bar(df_ml_imp, x='중요도', y='특성', orientation='h', title='머신러닝(RF) 특성 중요도 분석', text_auto='.4f', color_discrete_sequence=['#1DB954'])
fig_ml_imp.write_image(f'{image_dir}ml_importance.png')
fig_ml_imp.show()

# 딥러닝(DNN) 중요도 시각화 - 신뢰감 있는 파란색 테마
df_dl_imp = pd.DataFrame({'특성': X_test.columns, '중요도': r_dl.importances_mean}).sort_values(by='중요도', ascending=True)
fig_dl_imp = px.bar(df_dl_imp, x='중요도', y='특성', orientation='h', title='딥러닝(DNN) 특성 중요도 분석', text_auto='.4f', color_discrete_sequence=['#3498DB'])
fig_dl_imp.write_image(f'{image_dir}dl_importance.png')
fig_dl_imp.show()

# 각 모델의 예측 확률과 예측값을 구합니다
y_prob_ml = ml_model.predict_proba(X_test)[:, 1]
ml_thresh = metrics.get('RandomForest', {}).get('Best Threshold', 0.5)
y_pred_ml = (y_prob_ml >= ml_thresh).astype(int)
y_pred_dl = dl_model_wrapped.predict(X_test)

# 혼동 행렬을 모델별로 생성하여 저장합니다
for y_pred, name, file, col in zip([y_pred_ml, y_pred_dl], ['머신러닝(RF)', '딥러닝(DNN)'], ['ml_cm.png', 'dl_cm.png'], ['Greens', 'Blues']):
    cm = confusion_matrix(y_test, y_pred)
    fig_cm = px.imshow(cm, text_auto=True, color_continuous_scale=col, x=['유지', '이탈'], y=['유지', '이탈'], title=f'{name} 혼동 행렬')
    fig_cm.update_layout(xaxis_title='예측된 상태', yaxis_title='실제 상태')
    fig_cm.write_image(f'{image_dir}{file}')
    fig_cm.show()

# ROC 및 PR 커브를 각 모델별로 분리하여 점수와 함께 저장합니다
X_test_dl_proc = dl_prep.transform(X_test).astype(np.float32)
y_prob_dl = ort_session.run(None, {ort_session.get_inputs()[0].name: X_test_dl_proc})[0].flatten()

for prob, label, roc_file, pr_file, color in zip(
    [y_prob_ml, y_prob_dl], ['머신러닝(RF)', '딥러닝(DNN)'], 
    ['ml_roc.png', 'dl_roc.png'], ['ml_pr.png', 'dl_pr.png'], ['#1DB954', '#3498DB']
):
    # ROC 커브 생성
    fpr, tpr, _ = roc_curve(y_test, prob)
    roc_score = auc(fpr, tpr)
    fig_roc = go.Figure()
    fig_roc.add_trace(go.Scatter(x=fpr, y=tpr, name=f'AUC = {roc_score:.4f}', line=dict(color=color, width=3)))
    fig_roc.add_shape(type='line', line=dict(dash='dash'), x0=0, x1=1, y0=0, y1=1)
    fig_roc.update_layout(title=f'{label} ROC 커브 (AUC: {roc_score:.4f})', xaxis_title='거짓 양성률(FPR)', yaxis_title='참 양성률(TPR)')
    fig_roc.write_image(f'{image_dir}{roc_file}')
    
    # PR 커브 생성
    prec, rec, _ = precision_recall_curve(y_test, prob)
    pr_score = average_precision_score(y_test, prob)
    fig_pr = go.Figure()
    fig_pr.add_trace(go.Scatter(x=rec, y=prec, name=f'PR-AUC = {pr_score:.4f}', line=dict(color=color, width=3)))
    fig_pr.update_layout(title=f'{label} 정밀도-재현율 커브 (점수: {pr_score:.4f})', xaxis_title='재현율(Recall)', yaxis_title='정밀도(Precision)')
    fig_pr.write_image(f'{image_dir}{pr_file}')

print("✅ 모든 시각화 결과가 모델별로 개별 파일에 저장되었습니다.")

두 모델의 특성 중요도를 계산하고 있습니다. 데이터 양에 따라 시간이 조금 소요됩니다...


✅ 모든 시각화 결과가 모델별로 개별 파일에 저장되었습니다.
